# Trabalho Prático 02

- __Aluno__: Thiago Martin Poppe
- __Matrícula__: 2017014324

In [1]:
import numpy as np
import networkx as nx

import matplotlib.pyplot as plt

In [2]:
def generate_points(lower=0, upper=100, size=4):
    """
        Função que gera pontos com coordenadas inteiras em R²
        
        Parâmetros:
        ----------
        lower : int (opcional)
            O menor valor que pode ser gerado pelo gerador (por padrão é 0)
            
        upper : int (opcional)
            O maior valor que pode ser gerado pelo gerador (por padrão é 10)
            
        size : int (opcional)
            Número de pontos a serem gerados. Iremos gerar 2^size pontos, onde size
            deve ser um valor no intervalo [4, 10] (por padrão é 4)
            
        Retorno:
        -------
        Caso o valor de size seja inviável, a função retornará None. Senão, retornará uma
        lista de pontos com coordenadas inteiras em R²
            
    """
    
    # Verificando se o parâmetro size é viável
    if size < 4 or size > 10:
        print('*** O parâmetro size deve ter valor entre [4, 10] ***')
        return None
    
    # Gerando 2**size pontos inteiros
    points = []
    for i in range(2**size):
        p = np.random.randint(lower, upper, 2)
        points.append(p)
        
    return points

In [3]:
def get_euclidian_distance(points):
    """
        Função que calcula a distância euclidiana entre todos os pontos
        
        Parâmetros:
        ----------
        points : list of numpy.ndarray
            Lista contendo os pontos no plano R²
            
        Retorno:
        -------
        A função retorna uma lista de tuplas, onde o primeiro e segundo elementos são
        os vértices e o terceiro a distância calculada entre os mesmos usando a distância
        euclidiana
    """
    
    edges = []
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            dist = np.linalg.norm(points[i] - points[j])
            edges.append((i, j, dist))
            
    return edges

In [4]:
def get_manhattan_distance(points):
    """
        Função que calcula a distância manhattan entre todos os pontos
        
        Parâmetros:
        ----------
        points : list of numpy.ndarray
            Lista contendo os pontos no plano R²
            
        Retorno:
        -------
        A função retorna uma lista de tuplas, onde o primeiro e segundo elementos são
        os vértices e o terceiro a distância calculada entre os mesmos usando a distância
        manhattan
    """
    
    edges = []
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            dist = np.abs(points[i] - points[j]).sum()
            edges.append((i, j, dist))
            
    return edges

In [5]:
def twice_around_the_tree(graph):
    """
        Função que implementa o algoritmo 2-aproximativo para o problema do TSP
        
        Parâmetros:
        ----------
        graph : grafo
            Grafo de entrada para o nosso problema
            
        Retorno:
        -------
        A função retorna a ordem dos vértices do caminho aproximado e o seu tamanho
    """
    
    # Encontrando a árvore geradora mínima do grafo
    MST = nx.minimum_spanning_tree(G)
    
    # Caminhando em pré-ordem pela árvore usando o vértice 0 como raiz e fechando o ciclo
    # Hamiltoniano, conectando o vértice final ao inicial
    walk = list(nx.dfs_preorder_nodes(MST, source=0))
    walk.append(0)

    # Computando o tamanho do caminho encontrado
    length = 0
    for i in range(len(walk)-1):
        u, v = walk[i], walk[i+1]
        length += G[u][v]['weight']
    
    return walk, length    

In [6]:
def christofides(graph):
    """
        Função que implementa o algoritmo 1.5-aproximativo para o problema do TSP
        
        Parâmetros:
        ----------
        graph : grafo
            Grafo de entrada para o nosso problema
            
        Retorno:
        -------
        A função retorna a ordem dos vértices do caminho aproximado e o seu tamanho
    
    """
    
    # Copiando o grafo para não modificarmos o original
    G = graph.copy()
    
    # Encontrando a árvore geradora mínima do grafo
    MST = nx.minimum_spanning_tree(G)
    
    # Criando o conjunto de vértices que possuem grau ímpar e montando um subgrafo induzido a partir dos mesmos
    odd_degree_nodes = []
    for node in MST.nodes():
        if MST.degree(node) % 2 == 1:
            odd_degree_nodes.append(node)
    
    induced_graph = G.subgraph(odd_degree_nodes)
    
    # Multiplicando por -1 os pesos do grafo induzido
    for (u, v) in induced_graph.edges():
        induced_graph[u][v]['weight'] **= -1
        
    # Encontrando o matching perfeito de peso mínimo e voltando os pesos para o original
    min_weight_matching = nx.max_weight_matching(induced_graph, maxcardinality=True)
    
    # Retornando as arestas para o valor original
    for (u, v) in induced_graph.edges():
        induced_graph[u][v]['weight'] **= -1
    
    # Criando um subgrafo induzido com as arestas de matching de peso mínimo
    min_weight_matching_graph = G.edge_subgraph(min_weight_matching)
        
    # Criando um multigrafo com os vértices de G e arestas da MST e do matching perfeito de peso mínimo
    multigraph = nx.MultiGraph()
    multigraph.add_weighted_edges_from(MST.edges.data('weight'))
    multigraph.add_weighted_edges_from(min_weight_matching_graph.edges.data('weight'))
    
    # Computando o circuito euleriano
    eulerian_circuit = [u for (u, v) in nx.eulerian_circuit(multigraph, source=0)]
    
    # Retirando vértices repetidos, construindo assim um circuito hamiltoniano
    walk = []
    for node in eulerian_circuit:
        if node not in walk:
            walk.append(node)
    
    walk.append(0)
    
    # Computando o tamanho do caminho encontrado
    length = 0
    for i in range(len(walk)-1):
        u, v = walk[i], walk[i+1]
        length += G[u][v]['weight']
        
    return walk, length

In [7]:
# Exemplo tirado do site: https://www.brainkart.com/article/Approximation-Algorithms-for-the-Traveling-Salesman-Problem_8065/
edges = [
    (0, 1, 4),
    (0, 2, 8),
    (0, 3, 9),
    (0, 4, 12),
    
    (1, 2, 6),
    (1, 3, 8),
    (1, 4, 9),
    
    (2, 3, 10),
    (2, 4, 11),
    
    (3, 4, 7)
]

G = nx.Graph()
G.add_weighted_edges_from(edges)

In [8]:
christofides(G)

([0, 1, 3, 4, 2, 0], 38.0)

In [9]:
twice_around_the_tree(G)

([0, 1, 2, 3, 4, 0], 39)

In [10]:
# Exemplo tirado do Wikipédia: https://en.wikipedia.org/wiki/Christofides_algorithm
edges = [
    (0, 1, 1),
    (0, 2, 1),
    (0, 3, 2),
    (0, 4, 1),
    
    (1, 2, 2),
    (1, 3, 1),
    (1, 4, 1),
    
    (2, 3, 1),
    (2, 4, 1),
    
    (3, 4, 1)
]

G = nx.Graph()
G.add_weighted_edges_from(edges)

In [11]:
christofides(G)

([0, 4, 2, 3, 1, 0], 5.0)

In [12]:
twice_around_the_tree(G)

([0, 1, 3, 2, 4, 0], 5)